<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Adversarial Search (25 Points)

HW3 :: Practical Question 1

<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan


Consider the given $8\times8$ board game. There are two players in this game and they try to block each other and limit their valid moves by making walls. Their moves are limited to the 8 cells (or less when they are around the edges) beside them and the walls will start to appear only when a player steps into a random generated yellow cell and the wall will continue to be built until the player reaches the next yellow cell. The players start the game at $(0, 0)$ and $(7, 7)$ and they keep playing until one of them loses. There are multiple ways for a player to win the match:

- If a player steps into a wall or out of the board, the player loses the match!
- If the players reach the maximum moves the game finishes and the winner would be chosen based on the maximum valid move they have left plus the number of walls they have made and "No more moves" massege will appear on terminal.
- If the getMove() function does not return any value after 2 seconds, the player loses the match and "Time limit exceeded" massage will appear on terminal.

<center>
<img src="assets/Q3.png" style="width: 50%"/>
</center>


In this question, the codes for starting the game were provided for you in Helper-codes folder. You are expected to implement the MinimaxPlayer class (which inherit the Player class) with alpha-beta pruning.

In [98]:
%pip install tk


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [99]:
import numpy as np
import random
import time

from copy import deepcopy

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


You can run cell below and watch two naive players.

In [100]:
p1 = NaivePlayer(1, 0, 0) # blue
p2 = NaivePlayer(2, 7, 7) # red

g = Game(p1, p2)

numberOfMatches = 1
score1, score2 = g.start(numberOfMatches)

print(score1 / numberOfMatches)


Player 2 has made an invalid move
Player 1 has won

1.0


In [101]:
class MinimaxPlayer(Player):
    def __init__(self, col, x, y, max_depth=7):
        super().__init__(col, x, y)
        self.max_depth = max_depth
        self.us = col
        self.enemy = col
        if col == 1:
            self.enemy = 2
        else:
            self.enemy = 1

    ##############################
    ## Feel free to add methods ##
    ##############################

    def min_value(self, board, alpha, beta, depth):
        ##########################
        # Code Here! (10 Points) #
        # Use alpha-beta pruning #
        ##########################
        if depth == self.max_depth:
            return board.getScore(self.us) - board.getScore(self.enemy), -1
        
        cur_x = board.getPlayerX(self.enemy)
        cur_y = board.getPlayerY(self.enemy)

        best_direction = 0
        best_value = 1000

        for i in range(4):
            new_x = cur_x
            new_y = cur_y
            value = 1000
            if i == 0:
                new_x = new_x + 1
            elif i == 1:
                new_x = new_x - 1
            elif i == 2:
                new_y = new_y + 1
            else:
                new_y = new_y - 1

            new_board = deepcopy(board)
            res = new_board.move(IntPair(new_x, new_y), self.enemy)
            if res == -1:
                value = 1000
            elif res == -2:
                value = new_board.getScore(self.us) - new_board.getScore(self.enemy)
            else:
                value, direction = self.max_value(new_board, alpha, beta, depth + 1)

            if value < best_value:
                best_value = value
                best_direction = i
            beta = min(beta, best_value)
            if beta <= alpha:
                break

        return best_value, best_direction
        pass

    def max_value(self, board, alpha, beta, depth):
        ##########################
        # Code Here! (10 Points) #
        # Use alpha-beta pruning #
        ##########################
        if depth == self.max_depth:
            return board.getScore(self.us) - board.getScore(self.enemy), -1
        
        cur_x = board.getPlayerX(self.us)
        cur_y = board.getPlayerY(self.us)

        best_direction = 0
        best_value = -1000

        for i in range(4):
            new_x = cur_x
            new_y = cur_y
            value = -1000
            if i == 0:
                new_x = new_x + 1
            elif i == 1:
                new_x = new_x - 1
            elif i == 2:
                new_y = new_y + 1
            else:
                new_y = new_y - 1

            new_board = deepcopy(board)
            res = new_board.move(IntPair(new_x, new_y), self.us)
            if res == -1:
                value = -1000
            elif res == -2:
                value = new_board.getScore(self.us) - new_board.getScore(self.enemy)
            else:
                value, direction = self.min_value(new_board, alpha, beta, depth + 1)
            if value > best_value:
                best_value = value
                best_direction = i
            alpha = max(alpha, best_value)
            if beta <= alpha:
                break

        return best_value, best_direction
        pass

    def getMove(self, board):
        ################################################################################
        # Code Here! (5 Points)                                                        #
        # return IntPair(10, 10) when time limit exceeded or there is no possible move #
        # return IntPair(20, 20) when max number of moves exceeded                     #
        ################################################################################
        best_val, best_dir = self.max_value(board, -1000, 1000, 0)
        x_next = self.getX()
        y_next = self.getY()
        start = time.time()
        while ((x_next == self.getX()) and (y_next == self.getY()) ):
            rnd = best_dir
            if ((rnd == 0) and (self.getX() + 1 < board.getSize()) and (board.getCell(self.getX() + 1, self.getY()).getColor() == 0)):
                x_next += 1
            elif ((rnd == 1) and (self.getX() - 1 >= 0) and (board.getCell(self.getX() - 1, self.getY()).getColor() == 0)):
                x_next -= 1
            elif ((rnd == 2) and (self.getY() + 1 < board.getSize()) and (board.getCell(self.getX(), self.getY() + 1).getColor() == 0)):
                y_next += 1
            elif ((rnd == 3) and (self.getY() - 1 >= 0) and (board.getCell(self.getX(), self.getY() - 1).getColor() == 0)):
                y_next -= 1
            
        return IntPair(x_next, y_next)
        pass


In [104]:
p1 = MinimaxPlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

number_of_matches = 1
score1, score2 = g.start(number_of_matches)

print(score1 / number_of_matches)

Player 2 has exceeded the time limit
Player 1 has won

1.0
